##You will need a CUDA enabled GPU installed to run this code##

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import ast
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the CSV file
labels_df = pd.read_csv('encoded_labels.csv')

# Columns in your CSV
file_column = 'index'  # Replace with your actual filename column name
label_column = 'labels'  # Replace with your actual labels column name

# Directory where your images are stored
image_directory = './images/'  # Ensure this path ends with a '/'

# Function to convert string representation to list of integers
def parse_labels(label_str):
    # Replace spaces with commas to create a valid Python list string
    label_str = label_str.replace(' ', ',')
    # Use ast.literal_eval to safely evaluate the string to a list
    return list(map(int, ast.literal_eval(label_str)))

# Create full paths to images and parse labels
image_paths = [os.path.join(image_directory, fname) for fname in labels_df[file_column]]
labels = [parse_labels(label) for label in labels_df[label_column]]

# Split the dataset into training and validation sets
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    image_paths,
    labels,
    test_size=0.2,  # 20% for validation
    random_state=42
)

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.float)  # Multi-label classification

        image = Image.open(image_path).convert('RGB')  # Convert grayscale to RGB
        if self.transform:
            image = self.transform(image)

        return image, label

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize to 224x224, standard input size for ResNet
    transforms.ToTensor(),                        # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Create datasets and dataloaders
train_dataset = CustomDataset(train_image_paths, train_labels, transform=transform)
val_dataset = CustomDataset(val_image_paths, val_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the modified ResNet-50 model for grayscale images
class ModifiedResNet(nn.Module):
    def __init__(self):
        super(ModifiedResNet, self).__init__()
        self.resnet = resnet50(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 14)  # Number of classes
    
    def forward(self, x):
        return self.resnet(x)

model = ModifiedResNet().to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matt\anaconda3\envs\final_project\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Loss: 0.19373972814345192
Epoch 2, Loss: 0.18269072642140355
Epoch 3, Loss: 0.17879800540758362
Epoch 4, Loss: 0.17785667213248024
Epoch 5, Loss: 0.17585511859638472
Epoch 6, Loss: 0.1744772382608965
Epoch 7, Loss: 0.17419039109285842
Epoch 8, Loss: 0.17447147835442361
Epoch 9, Loss: 0.1727143205332418
Epoch 10, Loss: 0.17239151094505129
Validation Accuracy: 1328.609625668449%


In [5]:
# Validation loop
model.eval()
total_samples = len(val_loader.dataset)
correct_samples = torch.zeros(14, dtype=torch.int)  # Assuming 14 labels/classes

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        predictions = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
        predicted_labels = (predictions > 0.5).float()

        # For each label, count how many labels are correctly predicted
        for i in range(14):  # Loop over the 14 pathologies
            correct_samples[i] += (predicted_labels[:, i] == labels[:, i]).sum().item()

# Calculate accuracy for each class
accuracy_per_class = correct_samples / total_samples
average_accuracy = accuracy_per_class.mean().item()

print(f'Validation Accuracy per class: {accuracy_per_class.tolist()}')
print(f'Average Validation Accuracy: {100 * average_accuracy}%')

Validation Accuracy per class: [0.893048107624054, 0.9795008897781372, 0.9554367065429688, 0.9723707437515259, 0.8832442164421082, 0.9759358167648315, 0.98128342628479, 0.9973261952400208, 0.8262032270431519, 0.9509803652763367, 0.9483066201210022, 0.9777183532714844, 0.9893048405647278, 0.9554367065429688]
Average Validation Accuracy: 94.90069150924683%


In [6]:
torch.save(model.state_dict(), 'resnet50_scratch.pth')
